# Sequential pattern mining(SPM) - Rules extraction

Using the extracted **frequent sequential patterns**, filters and keeps only the partners **ending in the target event**. After this it makes a dictionary with the rules(patterns) that have **great support**. Finally, continuously check whether **any pattern** in dictionary **applies** to **testing dataset** and displays an appropriate message.

### Setup

In [1]:
#Make the necessary imports.

import re, copy,sys
from collections import deque

### Init variables

In [2]:
ifile = ifile = open("C:/Users/Public/ptyxiakh/training_my_dataset2_results2.csv", "r") #open(sys.argv[1],"r")

target_event = 11 #target event
target_event_sup = 3 #support of the target event
rules_list = [] #empty list

### Extract the rule list

In [13]:
i = 0
for line in ifile: #for each line of the .csv file
    rule = deque([])
    line_split = line.replace(" ","").strip().split("-1") #split the line based on -1 and change " " to ""
    STORE = True #variable that shows if the event is the target event
    pre_day = -1 #previous day
    
    for token in line_split: #for every token of the splited line
        m = re.search('\<([0-9]+)\>([0-9]+)', token) #take the numbrer, which is inside < >
        if(m is not None): 
            day = int(m.group(1))
            if(pre_day != -1): #if there is a previous day
                day = int(m.group(1))-pre_day
                
            pre_day = int(m.group(1)) 
            event = m.group(2)
            rule.append((day,event))
            
            STORE = (event == str(target_event)) #True if event is the target event, else False
            
        m = re.search('\#SUP:([0-9]+)', token) #take the number, which is after #
        if(m is not None):
            sup = m.group(1) #support of the pattern
            STORE = (STORE and int(sup) >= int(target_event_sup)) #True if STORE and suppurt of pattern >= support of target event
            
    if(STORE):
        rules_list.append(rule) #append the rule at the rule list
        i += 1

rules_list = sorted(rules_list, key=len) #sorted by the length of the rules


print("The rule list is:")
for i in range(0,len(rules_list)):
    print(rules_list[i])

The rule list is:
deque([(0, '3'), (4, '11')])
deque([(0, '4'), (4, '11')])
deque([(0, '47'), (4, '11')])
deque([(0, '45'), (4, '11')])
deque([(0, '457'), (4, '11')])
deque([(0, '5'), (4, '11')])
deque([(0, '57'), (4, '11')])
deque([(0, '7'), (4, '11')])


### Change rules from list to dictionary 

In [14]:
rules_dict = {} #dictionary for the rules

for rule in rules_list:# for every rule in rule list
    first_event = rule[0][1] #first event of the rule
    
    #append(add) rule to dicrionary
    if(first_event not in rules_dict.keys()):
        rule.popleft()
        rules_dict[first_event] = [rule] 
    else:
        rule.popleft()
        rules_dict[first_event].append(rule)

print("The rule keys are:")
print(rules_dict.keys())

ifile.close()

The rule keys are:
dict_keys(['3', '4', '47', '45', '457', '5', '57', '7'])


## Predict failure based on the rules

### Init the necessary variables

In [17]:
day_cnt = 0 #timestamp(time) counter
ep_count=1 #counter for the episodes
ifile = open("C:/Users/Public/ptyxiakh/testing_my_dataset2_spm.csv", "r") #open(sys.argv[2],"r") #path to spm testing dataset 

### Check for the pattern and make warnings

In [18]:
for line in ifile: #for every line of the file(for each episode)
    
    print("-------------------------")
    print("For episode",ep_count) #for every episode
    print("-------------------------")
    
    ep_count+=1 
    line = line.replace("-2","").split("-1") #split the line based on -2 and change "-2" to ""
    day_cnt = 0 
    
    for day in line: #for every day of the episode
        
        events = re.findall("\ ([0-9]+)",day) #take the events of the day
        if(len(events)==0): continue #if there is no events continue with the next episode
            
        for event in events: #for every event of the day
            
            if(str(event) == str(target_event)): #if the event is the target event
                print (str(day_cnt) + "> Failure") #print a Failure message
                
            if(str(event) in rules_dict.keys()): #if the event is in the keys of rules' dicitonary
                
                rules_to_activate = copy.deepcopy(rules_dict[str(event)]) #check which rules should be enabled
                
                if rules_to_activate:
                    for rule in rules_to_activate and rules_to_activate: #for every rule
                        if rule:
                            
                            act_rule = rule.popleft()
                            act_day = act_rule[0] #act day
                            act_event = act_rule[1] #act event
                            
                            if(str(act_event) == str(target_event)): #if act event is the target evet
                                
                                #print a Warning message
                                print (str(day_cnt)+"> Warning in " + str(act_day) + " days expect a fault!, cause of rule: "+str(event)+" -> "+(str(act_event)))
                                continue

        day_cnt += 1


ifile.close()

-------------------------
For episode 1
-------------------------
0> Warning in 4 days expect a fault!, cause of rule: 3 -> 11
0> Warning in 4 days expect a fault!, cause of rule: 4 -> 11
0> Warning in 4 days expect a fault!, cause of rule: 5 -> 11
0> Warning in 4 days expect a fault!, cause of rule: 5 -> 11
0> Warning in 4 days expect a fault!, cause of rule: 7 -> 11
1> Warning in 4 days expect a fault!, cause of rule: 4 -> 11
1> Warning in 4 days expect a fault!, cause of rule: 5 -> 11
1> Warning in 4 days expect a fault!, cause of rule: 7 -> 11
2> Warning in 4 days expect a fault!, cause of rule: 3 -> 11
2> Warning in 4 days expect a fault!, cause of rule: 5 -> 11
2> Warning in 4 days expect a fault!, cause of rule: 4 -> 11
3> Warning in 4 days expect a fault!, cause of rule: 3 -> 11
3> Warning in 4 days expect a fault!, cause of rule: 4 -> 11
3> Warning in 4 days expect a fault!, cause of rule: 5 -> 11
3> Warning in 4 days expect a fault!, cause of rule: 7 -> 11
3> Warning in 4 day